This code is used to find the optimal Arima Parameters

Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

## Read Data Files, format them properly

Read Data

In [3]:
df = pd.read_excel("../data/raw_data/data_clean3.xlsx")
df.head()

,Trm Code,Year,Academic Year,Month Start,Term,Course Subject,College (imperfect mapping),Course Subject and Number,Primary Instructor Name,Section,...,B,C,D,F,S,U,V,I,W,IJ
0,202208,2022,2022-23,8,Fall,ACCT,Scheller College of Business,ACCT 2101,"Blunck, Ryan",E,...,0.231884,0.144928,NaN,0.072464,NaN,NaN,NaN,NaN,0.014493,NaN
1,202105,2021,2021-22,5,Summer,ACCT,Scheller College of Business,ACCT 2101,"Blunck, Ryan",ES,...,0.060606,0.030303,NaN,0.060606,NaN,NaN,0.030303,NaN,0.030303,NaN
2,201902,2019,2018-19,2,Spring,ACCT,Scheller College of Business,ACCT 2101,"Blunck, Ryan",A,...,0.235294,0.058824,0.044118,0.014706,0.014706,NaN,NaN,NaN,0.044118,NaN
3,201408,2014,2014-15,8,Fall,ACCT,Scheller College of Business,ACCT 2101,"Carlisle, Melissa",TSB,...,0.413793,0.137931,0.068966,NaN,NaN,NaN,NaN,NaN,0.172414,NaN
4,201408,2014,2014-15,8,Fall,ACCT,Scheller College of Business,ACCT 2101,"Carlisle, Melissa",TSC,...,0.371429,0.200000,0.085714,0.028571,NaN,NaN,0.028571,NaN,0.085714,NaN


### Format the dataset: encode terms, split into term data, fill nans ...

Create New Dataframe

In [4]:
# list of all courses
courses_list = df["Course Subject and Number"].unique().tolist()

cols = ['2014_Spring', '2014_Summer', '2014_fall', '2015_Spring', '2015_Summer', '2015_fall', '2016_Spring', '2016_Summer', '2016_fall', 
        '2017_Spring', '2017_Summer', '2017_fall', '2018_Spring', '2018_Summer', '2018_fall', '2019_Spring', '2019_Summer', '2019_fall', 
        '2020_Spring', '2020_Summer', '2020_fall', '2021_Spring', '2021_Summer', '2021_fall', '2022_Spring', '2022_Summer', '2022_fall', '2023_Spring', '2023_Summer']

# create an empty dataframe to store courses' average grades over time from 2014 spring to 2023 summer
empty_df = pd.DataFrame(columns=cols)


# change Fall to fall (so values sort as Spring Summer fall)
change_fall = df.copy()
change_fall = change_fall.replace({'Term': 'Fall'}, {'Term': 'fall'})

# populate the empty_df
for course in courses_list:

#     # get only that course
#     course_df = change_fall[change_fall["Course Subject and Number"] == course]


#     # group by semester
#     new_df = course_df.groupby(["Year", "Term"]).mean().drop(columns=['Trm Code', 'Month Start', 'A', 'B', 'C', 'D', 'F', 'S', 'U', 'V', 'I', 'W', 'IJ'])

#     # unnecessary but good for debugging
#     new_df = new_df.reset_index(drop=False)
#     new_df['time'] = new_df['Year'].astype(str) + "_" + new_df['Term']
#     # print(new_df['time'])

#     # add to dataframe
#     new_df = new_df.set_index('time')
#     new_df = new_df.drop(columns=['Year', 'Term'])
#     new_df = new_df.rename(columns={'Average Grade': course})
#     new_df = new_df.transpose()
#     new_df.columns.name = ''

#     empty_df = pd.concat([empty_df, new_df])

        # Get only that course
    course_df = change_fall[change_fall["Course Subject and Number"] == course]

    # Convert relevant columns to numeric
    numeric_cols = ['Average Grade']  # Add other columns if needed
    course_df[numeric_cols] = course_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Group by semester and calculate the mean
    new_df = course_df.groupby(["Year", "Term"])[numeric_cols].mean().reset_index()

    # Create a 'time' column for indexing
    new_df['time'] = new_df['Year'].astype(str) + "_" + new_df['Term']

    # Add to the dataframe
    new_df = new_df.set_index('time')
    new_df = new_df.drop(columns=['Year', 'Term'])
    new_df = new_df.rename(columns={'Average Grade': course})
    new_df = new_df.transpose()
    new_df.columns.name = ''

    empty_df = pd.concat([empty_df, new_df])


/var/folders/n8/cpfk46ss313d61dgywbkx80m0000gn/T/ipykernel_18560/449484121.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df[numeric_cols] = course_df[numeric_cols].apply(pd.to_numeric, errors='coerce')
/var/folders/n8/cpfk46ss313d61dgywbkx80m0000gn/T/ipykernel_18560/449484121.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_df[numeric_cols] = course_df[numeric_cols].apply(pd.to_numeric, errors='coerce')
/var/folders/n8/cpfk46ss313d61dgywbkx80m0000gn/T/ipykernel_18560/449484121.py:45

In [5]:
empty_df.head()

,2014_Spring,2014_Summer,2014_fall,2015_Spring,2015_Summer,2015_fall,2016_Spring,2016_Summer,2016_fall,2017_Spring,...,2020_Summer,2020_fall,2021_Spring,2021_Summer,2021_fall,2022_Spring,2022_Summer,2022_fall,2023_Spring,2023_Summer
ACCT 2101,3.136000,3.29,3.038000,3.210000,3.21,2.998000,3.086000,3.00,3.216,3.136000,...,3.69,3.452500,3.036000,3.61,3.175000,2.832500,3.82,3.176000,3.2275,3.00
ACCT 2102,2.770000,3.27,3.120000,3.150000,3.55,3.290000,3.156667,3.11,3.160,3.226667,...,2.86,3.666667,3.566667,3.08,3.056667,3.673333,NaN,3.383333,3.5600,3.33
AE 1350,3.336667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AE 1355,4.000000,NaN,3.956667,3.900000,NaN,4.000000,4.000000,NaN,3.955,4.000000,...,NaN,4.000000,4.000000,NaN,4.000000,4.000000,NaN,4.000000,4.0000,NaN
AE 1601,NaN,NaN,3.596667,3.746667,NaN,3.626667,3.783333,NaN,3.900,3.733333,...,4.00,3.826667,3.750000,NaN,3.863333,3.820000,NaN,3.930000,3.9000,NaN


 Rename columns in order to make time series analysis easier

In [6]:
testy_df = empty_df.rename(columns={'2014_Spring': 1, '2014_Summer': 2, '2014_fall': 3, '2015_Spring': 4, '2015_Summer': 5, '2015_fall': 6, '2016_Spring': 7, '2016_Summer': 8,
                                    '2016_fall': 9, '2017_Spring': 10, '2017_Summer': 11, '2017_fall': 12, '2018_Spring': 13, '2018_Summer': 14, '2018_fall': 15, '2019_Spring': 16, 
                                    '2019_Summer': 17, '2019_fall': 18, '2020_Spring': 19, '2020_Summer': 20, '2020_fall': 21, '2021_Spring': 22, '2021_Summer': 23, '2021_fall': 24, 
                                    '2022_Spring': 25, '2022_Summer': 26, '2022_fall': 27, '2023_Spring': 28, '2023_Summer': 29})

# start by forecasting for courses with no NaN values
df_1 = testy_df.dropna()
df_2 = testy_df.copy()


Separate into seasons

In [7]:
df_spring = df_2.iloc[:,0::3]
df_summer = df_2.iloc[:,1::3]
df_fall = df_2.iloc[:, 2::3] 

## Impute Nan Values

Method:
- Up to the first non-nan value set them all equal to the first non-nan value you see on the right
- After that use linear interpolation between non-nan value to the left and right
- If no non-nan value to the right then just use the last non-nan value you see throughout

First drop rows if they are all nans

In [8]:
print(len(df_spring), len(df_summer), len(df_fall))

5092 5092 5092


In [9]:
df_spring = df_spring.dropna(how="all")
df_summer = df_summer.dropna(how="all")
df_fall = df_fall.dropna(how="all")

In [10]:
print(len(df_spring), len(df_summer), len(df_fall))

3429 1409 3157


Function to fill nans

In [11]:
def fill_nans(df):
    # Forward fill NaNs at the start of each row
    df_filled = df.apply(lambda x: x.ffill().bfill() if x.first_valid_index() is None else x, axis=1)
    
    # Apply linear interpolation for remaining NaNs
    df_filled = df_filled.interpolate(method='linear', axis=1, limit_direction='both')

    return df_filled

In [12]:
df_spring = fill_nans(df_spring)
df_summer = fill_nans(df_summer)
df_fall = fill_nans(df_fall)

In [13]:
print(df_spring.isna().any().sum(), df_summer.isna().any().sum(),df_fall.isna().any().sum())

0 0 0


Backup this for now

In [14]:
df_spring_backup = df_spring.copy()
df_summer_backup = df_summer.copy()
df_fall_backup = df_fall.copy()

In [15]:
df_spring_backup.index.nunique(), df_summer_backup.index.nunique(), df_fall_backup.index.nunique()

(3429, 1409, 3157)

Drop test columns which have nans : merge on teh right then resplit

In [16]:
df_spring = df_spring.merge(df_spring_test, left_index=True, right_index=True)
df_summer = df_summer.merge(df_spring_test, left_index=True, right_index=True)
df_fall = df_fall.merge(df_spring_test, left_index=True, right_index=True)

NameError: name 'df_spring_test' is not defined

In [ ]:
df_spring = df_spring.dropna(how="any")
df_summer = df_summer.dropna(how="any")
df_fall = df_fall.dropna(how="any")

Redo the split

In [ ]:
df_spring_test = df_spring.iloc[:,-1]
df_summer_test = df_summer.iloc[:,-1]
df_fall_test = df_fall.iloc[:,-1]

df_spring = df_spring.iloc[:,:-1]
df_summer = df_summer.iloc[:,:-1]
df_fall = df_fall.iloc[:,:-1]

Now let's train the modell

In [ ]:
print( len(df_spring.columns), len(df_summer.columns),len(df_fall.columns))

9 9 8


## Model Training

In [ ]:
df_spring.values.flatten()

array([3.136, 3.21 , 3.086, ..., 4.   , 4.   , 4.   ])

Rename indices to be in order

In [ ]:
df_spring_train = df_spring.copy()
df_summer_train = df_summer.copy()
df_fall_train = df_fall.copy()

df_spring_train.columns = [1,2,3,4,5,6,7,8,9]
df_summer_train.columns = [1,2,3,4,5,6,7,8,9]
df_fall_train.columns = [1,2,3,4,5,6,7,8]

df_spring_validation = df_spring_test.copy()
df_summer_validation = df_summer_test.copy()
df_fall_validation = df_fall_test.copy()

df_spring_validation.columns = [10]
df_summer_validation.columns = [10]
df_fall_validation.columns = [9]


Get list of courses and put everyting into list

In [ ]:
spring_courses = df_spring_train.index.to_list()
summer_courses = df_summer_train.index.to_list()
fall_courses = df_fall_train.index.to_list()

dfs = [df_spring_train, df_summer_train, df_fall_train]
tests = [df_spring_validation, df_summer_validation, df_fall_validation]
courses = [spring_courses, summer_courses, fall_courses]
prediction_indices = [10,10,9]

Get it working for a single course

In [ ]:
semester = 0 # 0 = spring, 1 = summer, 2 = fall

df = dfs[semester]
test = tests[semester]
course_list = courses[semester]
prediction_index = prediction_indices[semester]


# specify parameters
p,d,q = 1,1,2

# one for now
course_name = course_list[0]
# get all grades for this course and put into a dataframe
train_row = df.loc[course_name].tolist()

# ARIMA
model = ARIMA(train_row, order=(1,1,2)).fit()

# get predictions for all periods
pred1 = model.predict()

# since ARIMA starts at 1 pred1[0] is not calculated, use real grades as the starting point (for graph)
pred1[0] = train_row[0]

# next period's prediction
prediction = model.predict(prediction_index)

print("train predictions", pred1, " \n test predictions", prediction)

print("Train original values", train_row, "\n Test original values", test.loc[course_name].tolist())


print("MSE train data: ", mean_squared_error(train_row, pred1))
print("MSE test data: ", mean_squared_error([test.loc[course_name]], prediction))

train predictions [3.136      3.13599986 3.17859671 3.1302802  3.15109469 3.23476549
 3.19110574 3.33631802 3.16437921]  
 test predictions [3.17323667]
Train original values [3.136, 3.21, 3.086, 3.136, 3.498, 3.2019999999999995, 3.6480000000000006, 3.036, 2.8325] 
 Test original values 3.2275
MSE train data:  0.06050964997371952
MSE test data:  0.002944509147711605


# Training Loop to find best overall params

In [ ]:
best_params_per_semester = {}
min_mse_per_semester = {}

# Loop through each semester
optimize = False
if optimize == True:
    for semester in range(2,3):
        df = dfs[semester]
        test = tests[semester]
        course_list = courses[semester]
        prediction_index = prediction_indices[semester]

        best_params = None
        min_combined_mse = np.inf

        print("Now at semester ", semester)

        # Iterate over different ARIMA orders
        for p in range(1, 3):
            for d in range(1, 3):
                for q in range(1, 3):
                    total_train_mse = 0
                    total_test_mse = 0
                    valid_courses = 0

                    # Iterate over each course
                    for course_name in course_list:
                        try:
                            # Extract the training and testing series for this course
                            train_row = df.loc[course_name].tolist()
                            
                            # Fit the ARIMA model to this course's training data
                            model = ARIMA(train_row, order=(p, d, q)).fit()

                            # Predict 
                            train_pred = model.predict()
                            train_pred[0] = train_row[0] # fill in first value
                            

                            # Predict next step
                            prediction = model.predict(prediction_index) # no need for start end

                            # Compute MSEs
                            test_mse = mean_squared_error([test.loc[course_name]], prediction)
                            train_mse = mean_squared_error(train_row, train_pred)

                            # add errors
                            total_train_mse += train_mse
                            total_test_mse += test_mse
                            valid_courses += 1

                        except:
                            continue  # Skip if the model fails to fit

                    # Calculate combined MSE and update the best parameters if necessary
                    if valid_courses > 0:
                        combined_mse = total_train_mse + total_test_mse
                        if combined_mse < min_combined_mse:
                            min_combined_mse = combined_mse
                            best_params = (p, d, q)

        # Store the best parameters and combined MSE for this semester
        best_params_per_semester[semester] = best_params
        min_mse_per_semester[semester] = min_combined_mse

    # best_params_per_semester now holds the optimal (p, d, q) for each semester


In [ ]:
best_params_per_semester

{}